In [1]:
from ultralytics import YOLO

model = YOLO('./weights/last.pt')

# 알약 이미지 탐지

In [6]:
# 테두리 그리기
import datetime
import cv2
from ultralytics import YOLO

CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)

classcode = ['K-001562', 'K-001901', 'K-002052', 'K-004378', 'K-005391', 'K-006192', 'K-006563', 'K-006697', 'K-007024', 'K-007348', 'K-012769', 'K-013398', 'K-013900', 'K-015710', 'K-016206', 'K-016548', 'K-016688', 'K-021771', 'K-023933', 'K-024029', 'K-024127', 'K-025629', 'K-027777', 'K-028408', 'K-031490', 'K-038104', 'K-038722', 'K-040265', 'K-041149', 'K-044830', 'K-044834']
classname_en = ['Minocin Cap. 50mg', 'Buspar Tab. 10mg Boryung', 'Dinacs Tab.', 'Tylenol Tab. 500mg', 'Proscar Tab. 5mg', 'Beecom-CF Tab.', 'Joins Tab.', 'Ssiznal Tab.', 'Festal Plus Tab.', 'Riboteine Tab.', 'Dr. Bearse Tab.', 'Isotinone Soft Cap.', 'Ebixa Tab.', 'Entelon Tab. 150mg', 'Avodart Soft Cap. 0.5mg', 'Gabatopa Tab. 100mg', 'Omacor Soft Cap.', 'Rabiet Tab.20mg', 'Allerdin Tab.', 'Neotigason Cap. 10mg', 'Cacepin Tab 25mg', 'Zeromine Soft Cap.', 'Kanarb Tab. 60mg', 'Olzapine Tab. 2.5mg', 'Aspid Tab.', 'Hanmi Tams Cap. 0.4mg', 'Entresto Film Coated Tab. 100mg', 'Agen-F Tab.', 'Ramnos Cap. 500mg', 'Haben Herb F Cap.', 'Elsteine Tab.']


model = YOLO('./weights/last.pt')

image = './DATASET/알약 이미지 탐지.png'
frame = cv2.imread(image)
detection = model(image)[0] # results = model.predict('./DATASET/COMBI/K-001562_K-005391_K-005391_K-002052.png', )


for data in detection.boxes.data.tolist():  # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
    confidence = data[4]
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    x1, y1, x2, y2 = int(data[0]), int(data[1]), int(data[2]), int(data[3])
    cls = int(data[5])
    cv2.rectangle(frame, (x1, y1), (x2, y2), GREEN, 2)
    cv2.putText(frame, classname_en[cls]+' '+str(round(confidence, 2)) + '%', (x1, y1), cv2.FONT_ITALIC, 1, WHITE, 2)

cv2.imwrite("./temp/frame.jpg", frame)
cv2.imshow('frame', frame)
cv2.waitKey(0) 
cv2.destroyAllWindows()



# detection.boxes.data.tolist()
# [40.659400939941406, 910.831298828125, 224.98947143554688, 1102.4556884765625, 0.9819812178611755, 4.0]
# [685.2080078125, 930.0809326171875, 880.92236328125, 1119.1956787109375, 0.979022204875946, 2.0]
# [80.76461791992188, 73.91424560546875, 307.41168212890625, 452.0390625, 0.9753074645996094, 0.0]
# [528.9434814453125, 148.44329833984375, 920.2252197265625, 529.9608154296875, 0.9750985503196716, 4.0]

# [0] 40.659400939941406
# [1] 910.831298828125
# [2] 224.98947143554688
# [3] 1102.4556884765625 
# [4] 0.9819812178611755 conf
# [5] 4.0 cls



image 1/1 c:\Users\tomozart21\VS_PROJECT\PILL_RnE\DATASET\  .png: 640x512 2 K-001901s, 1 K-004378, 2 K-028408s, 1 K-031490, 1 K-044834, 65.0ms
Speed: 3.0ms preprocess, 65.0ms inference, 21.5ms postprocess per image at shape (1, 3, 640, 512)


In [12]:
# 병용금기 처리
import pandas as pd


classcode = ['K-001562', 'K-001901', 'K-002052', 'K-004378', 'K-005391', 'K-006192', 'K-006563', 'K-006697', 'K-007024', 'K-007348', 'K-012769', 'K-013398', 'K-013900', 'K-015710', 'K-016206', 'K-016548', 'K-016688', 'K-021771', 'K-023933', 'K-024029', 'K-024127', 'K-025629', 'K-027777', 'K-028408', 'K-031490', 'K-038104', 'K-038722', 'K-040265', 'K-041149', 'K-044830', 'K-044834']
classname = ['Minocin Cap. 50mg', 'Buspar Tab. 10mg Boryung', 'Dinacs Tab.', 'Tylenol Tab. 500mg', 'Proscar Tab. 5mg', 'Beecom-CF Tab.', 'Joins Tab.', 'Ssiznal Tab.', 'Festal Plus Tab.', 'Riboteine Tab.', 'Dr. Bearse Tab.', 'Isotinone Soft Cap.', 'Ebixa Tab.', 'Entelon Tab. 150mg', 'Avodart Soft Cap. 0.5mg', 'Gabatopa Tab. 100mg', 'Omacor Soft Cap.', 'Rabiet Tab.20mg', 'Allerdin Tab.', 'Neotigason Cap. 10mg', 'Cacepin Tab 25mg', 'Zeromine Soft Cap.', 'Kanarb Tab. 60mg', 'Olzapine Tab. 2.5mg', 'Aspid Tab.', 'Hanmi Tams Cap. 0.4mg', 'Entresto Film Coated Tab. 100mg', 'Agen-F Tab.', 'Ramnos Cap. 500mg', 'Haben Herb F Cap.', 'Elsteine Tab.']

class_dict = dict(zip(classcode, classname))

# 용도 : K-xxxxxx 코드 --> 648101510와 같은 코드로
df = pd.read_csv('./DRUG_INFO/선택한알약이미지정보.csv')
df_img = df[['drug_N', 'drug_code', 'dl_name', 'dl_name_en']].drop_duplicates(['drug_N']).reset_index(drop=True)

df = pd.read_csv('./DRUG_INFO/DUR_병용금기.csv')
df_mix_bann = df[['제품코드A', '제품명A', '업체명A', '제품코드B', '제품명B', '상세정보']]

df = pd.read_csv('./DRUG_INFO/DUR_연령별금기.csv')
df_age_bann = df[['성분명', '제품코드', '제품명', '특정연령', '특정연령단위-1', '특정연령단위-2', '상세정보']]

df = pd.read_csv('./DRUG_INFO/효능군중복주의약물.csv')
df_effect = df[['약품코드', '효능군', '일반명', '품목명', '업체명']]

C:\Users\tomozart21\AppData\Local\Temp\ipykernel_34228\1672492118.py:14: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('./DRUG_INFO/DUR_병용금기.csv')


In [3]:
# K-xxxxxx 코드 --> 648101510와 같은 코드로
df_img.head(2)

,drug_N,drug_code,dl_name,dl_name_en
0,K-001562,644700590,미노씬캡슐 50mg,Minocin Cap. 50mg
1,K-001901,641901270,보령부스파정 10mg,Buspar Tab. 10mg Boryung


In [4]:
# 병용금기 처리
df_mix_bann.head(2)

,제품코드A,제품명A,업체명A,제품코드B,제품명B,상세정보
0,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),653400910,케토라신정(케토롤락트로메타민)_(10mg/1정),중증의 위장관계 이상반응
1,648101510,에이서캡슐(아세클로페낙)_(0.1g/1캡슐),경동제약(주),59000681,케롤락주(케토롤락트로메타민염)_(30mg/1mL),중증의 위장관계 이상반응


In [24]:
df_age_bann.head(2)

,성분명,제품코드,제품명,특정연령,특정연령단위-1,특정연령단위-2,상세정보
0,acamprosate,651904740,명인아캄프로세이트정333mg(아캄프로세이트칼슘)_(0.333g/1정),65,세 (3),이상 (4),65세 이상에서 안전성 및 유효성 미확립. 연령이 높아질수록 신장기능이 저하되어 이...
1,acamprosate,657201040,환인아캄프로세이트정(아캄프로세이트칼슘)_(0.333g/1정),65,세 (3),이상 (4),65세 이상에서 안전성 및 유효성 미확립. 연령이 높아질수록 신장기능이 저하되어 이...


In [17]:
df_effect.head(2)

,약품코드,효능군,일반명,품목명,업체명
0,652100060,당뇨병용제,acarbose,글루코바이정100밀리그람(아카보즈)_(0.1g/1정),(주)한독
1,652100050,당뇨병용제,acarbose,글루코바이정50밀리그람(아카보즈)_(50mg/1정),(주)한독


### 알약 효능

In [41]:
import datetime
from PIL import ImageFont, ImageDraw, Image
import cv2
from ultralytics import YOLO


CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)

classcode = ['K-001562', 'K-001901', 'K-002052', 'K-004378', 'K-005391', 'K-006192', 'K-006563', 'K-006697', 'K-007024', 'K-007348', 'K-012769', 'K-013398', 'K-013900', 'K-015710', 'K-016206', 'K-016548', 'K-016688', 'K-021771', 'K-023933', 'K-024029', 'K-024127', 'K-025629', 'K-027777', 'K-028408', 'K-031490', 'K-038104', 'K-038722', 'K-040265', 'K-041149', 'K-044830', 'K-044834']
classname_en = ['Minocin Cap. 50mg', 'Buspar Tab. 10mg Boryung', 'Dinacs Tab.', 'Tylenol Tab. 500mg', 'Proscar Tab. 5mg', 'Beecom-CF Tab.', 'Joins Tab.', 'Ssiznal Tab.', 'Festal Plus Tab.', 'Riboteine Tab.', 'Dr. Bearse Tab.', 'Isotinone Soft Cap.', 'Ebixa Tab.', 'Entelon Tab. 150mg', 'Avodart Soft Cap. 0.5mg', 'Gabatopa Tab. 100mg', 'Omacor Soft Cap.', 'Rabiet Tab.20mg', 'Allerdin Tab.', 'Neotigason Cap. 10mg', 'Cacepin Tab 25mg', 'Zeromine Soft Cap.', 'Kanarb Tab. 60mg', 'Olzapine Tab. 2.5mg', 'Aspid Tab.', 'Hanmi Tams Cap. 0.4mg', 'Entresto Film Coated Tab. 100mg', 'Agen-F Tab.', 'Ramnos Cap. 500mg', 'Haben Herb F Cap.', 'Elsteine Tab.']


model = YOLO('./weights/last.pt')

image = './DATASET/약품 효능.png'
frame = cv2.imread(image)
detection = model(image)[0]

drug_list = []
boxes_dict = {}
ord_num = 1
for data in detection.boxes.data.tolist():  # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
    confidence = (data[4] * 100) / 2
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    x1, y1, x2, y2 = int(data[0]), int(data[1]), int(data[2]), int(data[3])
    cls = int(data[5])

    drug_list.append(classcode[cls])
    boxes_dict[ord_num] = [cls, classcode[cls], (x1, y1), (x2, y2), confidence]
    ord_num += 1

    # cv2.rectangle(frame, (x1, y1), (x2, y2), GREEN, 2)

print(boxes_dict)
font = ImageFont.truetype('./font/NanumGothic.ttf', 18)
image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
draw = ImageDraw.Draw(image, 'RGBA')

for i in range(1, len(boxes_dict)+1) :
    drug_N = boxes_dict[i][1]
    point1 = boxes_dict[i][2]
    point2 = boxes_dict[i][3]

    print(point1)
    drug1 = df_img.loc[df_img['drug_N']==drug_N, 'drug_code'].values[0]   # K-xxx --> 648101510
    result = df_effect.loc[df_effect['약품코드']==drug1, ['약품코드', '효능군', '일반명', '품목명', '업체명']]

    if len(result) >= 1 :
        name = result['품목명'].values[0]
        effect = result['효능군'].values[0]
        name = name.replace(')', ')\n')

       
        draw.rectangle((point1, point2), outline=(255, 0, 0), fill=(0, 0, 0, 50), width=3)
        #draw.text((point1), name+'\n'+effect, font=font, fill=(255, 0, 0, 0))
        draw.text((point1), name+'\n'+effect, font=font, fill=(255, 250, 0, 0))
        
    
    print(drug_N, len(result))


image.save('./temp/약품효능.jpg')


image 1/1 c:\Users\tomozart21\VS_PROJECT\PILL_RnE\DATASET\ .png: 640x512 1 K-001901, 1 K-024127, 1 K-027777, 1 K-028408, 1 K-038722, 1 K-044834, 51.5ms
Speed: 1.0ms preprocess, 51.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 512)


{1: [23, 'K-028408', (77, 815), (256, 993), 48.59852492809296], 2: [30, 'K-044834', (533, 66), (794, 329), 48.55341911315918], 3: [26, 'K-038722', (362, 962), (623, 1237), 48.22080433368683], 4: [1, 'K-001901', (144, 486), (401, 722), 48.11736345291138], 5: [22, 'K-027777', (563, 667), (733, 869), 47.962090373039246], 6: [20, 'K-024127', (151, 163), (287, 301), 47.68721163272858]}
(77, 815)
K-028408 1
(533, 66)
K-044834 1
(362, 962)
K-038722 1
(144, 486)
K-001901 1
(563, 667)
K-027777 1
(151, 163)
K-024127 1


# 알약 병용 금기

In [120]:
import datetime
from PIL import ImageFont, ImageDraw, Image
import cv2
from ultralytics import YOLO

def combination(drug_list) : 
    combi_list = []
    for i in range(len(drug_list)-1) :
        for j in range(i+1, len(drug_list)) :
            combi_list.append([drug_list[i], drug_list[j]])
    return combi_list

CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)

classcode = ['K-001562', 'K-001901', 'K-002052', 'K-004378', 'K-005391', 'K-006192', 'K-006563', 'K-006697', 'K-007024', 'K-007348', 'K-012769', 'K-013398', 'K-013900', 'K-015710', 'K-016206', 'K-016548', 'K-016688', 'K-021771', 'K-023933', 'K-024029', 'K-024127', 'K-025629', 'K-027777', 'K-028408', 'K-031490', 'K-038104', 'K-038722', 'K-040265', 'K-041149', 'K-044830', 'K-044834']
classname_en = ['Minocin Cap. 50mg', 'Buspar Tab. 10mg Boryung', 'Dinacs Tab.', 'Tylenol Tab. 500mg', 'Proscar Tab. 5mg', 'Beecom-CF Tab.', 'Joins Tab.', 'Ssiznal Tab.', 'Festal Plus Tab.', 'Riboteine Tab.', 'Dr. Bearse Tab.', 'Isotinone Soft Cap.', 'Ebixa Tab.', 'Entelon Tab. 150mg', 'Avodart Soft Cap. 0.5mg', 'Gabatopa Tab. 100mg', 'Omacor Soft Cap.', 'Rabiet Tab.20mg', 'Allerdin Tab.', 'Neotigason Cap. 10mg', 'Cacepin Tab 25mg', 'Zeromine Soft Cap.', 'Kanarb Tab. 60mg', 'Olzapine Tab. 2.5mg', 'Aspid Tab.', 'Hanmi Tams Cap. 0.4mg', 'Entresto Film Coated Tab. 100mg', 'Agen-F Tab.', 'Ramnos Cap. 500mg', 'Haben Herb F Cap.', 'Elsteine Tab.']


model = YOLO('./weights/last.pt')

image = './DATASET/병용금기2.png'
frame = cv2.imread(image)
detection = model(image)[0] # results = model.predict('./DATASET/COMBI/K-001562_K-005391_K-005391_K-002052.png', )

drug_list = []
boxes_dict = {}

ord_num = 1
for data in detection.boxes.data.tolist():  # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
    confidence = (data[4] * 100) / 2
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    x1, y1, x2, y2 = int(data[0]), int(data[1]), int(data[2]), int(data[3])
    cls = int(data[5])

    drug_list.append(classcode[cls])
    boxes_dict[ord_num] = [cls, classcode[cls], (x1, y1), (x2, y2), confidence]
    ord_num += 1

    cv2.rectangle(frame, (x1, y1), (x2, y2), GREEN, 2)
    cv2.putText(frame, classname_en[cls]+' ', (x1, y1-5), cv2.FONT_ITALIC, 1, WHITE, 2)
    # cv2.putText(frame, classname_en[cls]+' '+str(round(confidence, 2)) + '%', (x1, y1), cv2.FONT_ITALIC, 1, WHITE, 2)

pair_list = combination([i for i in range(1, len(boxes_dict)+1)]) # [[1, 2], [1, 3], [1, 4], ...]

alert_box = []
for pair in pair_list :
    drug1 = boxes_dict[pair[0]]  # 0, 1 --> K-xxx
    drug2 = boxes_dict[pair[1]]

    drug1 = df_img.loc[df_img['drug_N']==drug1[1], 'drug_code'].values[0]   # K-xxx --> 648101510
    drug2 = df_img.loc[df_img['drug_N']==drug2[1], 'drug_code'].values[0]

    result1 = df_mix_bann.loc[(df_mix_bann['제품코드A']==drug1) & (df_mix_bann['제품코드B']==drug2), '상세정보'].count()  # 병용금기 목록 확인
    result2 = df_mix_bann.loc[(df_mix_bann['제품코드A']==drug2) & (df_mix_bann['제품코드B']==drug1), '상세정보'].count()

    side_effect = ''
    if result1 == 1 :
        side_effect = df_mix_bann.loc[(df_mix_bann['제품코드A']==drug1) & (df_mix_bann['제품코드B']==drug2), '상세정보'].values[0]
        alert_box.append([pair, side_effect])
    if result2 == 1 :
        side_effect = df_mix_bann.loc[(df_mix_bann['제품코드A']==drug2) & (df_mix_bann['제품코드B']==drug1), '상세정보'].values[0]
        alert_box.append([pair, side_effect])



font = ImageFont.truetype('./font/NanumGothic.ttf', 45)
image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
draw = ImageDraw.Draw(image)
# image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
# frame = Image.fromarray(frame)
# draw = ImageDraw.Draw(frame)

for alert in alert_box :
    drug1 = boxes_dict[alert[0][0]]
    drug2 = boxes_dict[alert[0][1]]
    alert_msg = alert[1]
    print(drug1)



    # cv2.rectangle(frame, drug1[2], drug1[3], RED, 5)
    # cv2.rectangle(frame, drug2[2], drug2[3], RED, 5)

    # draw.rectangle(alert[2], outline=(0, 0, 255), width=5)
    draw.rectangle((drug1[2], drug1[3]), outline=(255, 0, 0), width=3)
    draw.rectangle((drug2[2], drug2[3]), outline=(255, 0, 0), width=3)

    x1, y1 = (drug1[3][0]-drug1[2][0])//2+drug1[2][0],  (drug1[3][1] - drug1[2][1])//2 + drug1[2][1]
    x2, y2 = (drug2[3][0]-drug2[2][0])//2+drug2[2][0],  (drug2[3][1] - drug2[2][1])//2 + drug2[2][1]


    draw.line((x1, y1,  x2, y2), fill="red", width=5)


    draw.text((drug1[2]), alert_msg, font=font, fill=(255, 0, 0, 0))

image.save('./temp/병용금기2.jpg')


image 1/1 c:\Users\tomozart21\VS_PROJECT\PILL_RnE\DATASET\2.png: 640x512 1 K-001562, 1 K-006697, 1 K-013398, 1 K-016206, 1 K-016688, 1 K-024029, 61.5ms
Speed: 2.0ms preprocess, 61.5ms inference, 2.0ms postprocess per image at shape (1, 3, 640, 512)


[11, 'K-013398', (17, 514), (256, 734), 48.22811782360077]
[19, 'K-024029', (614, 953), (927, 1241), 47.97317683696747]


# 연령대별 금기약

In [25]:
import pandas as pd

df = pd.read_csv('./DRUG_INFO/DUR_연령별금기.csv')
df_age_bann = df[['성분명', '제품코드', '제품명', '특정연령', '특정연령단위-2', '상세정보']]

df_age_bann.head(2)

,성분명,제품코드,제품명,특정연령,특정연령단위-2,상세정보
0,acamprosate,651904740,명인아캄프로세이트정333mg(아캄프로세이트칼슘)_(0.333g/1정),65,이상 (4),65세 이상에서 안전성 및 유효성 미확립. 연령이 높아질수록 신장기능이 저하되어 이...
1,acamprosate,657201040,환인아캄프로세이트정(아캄프로세이트칼슘)_(0.333g/1정),65,이상 (4),65세 이상에서 안전성 및 유효성 미확립. 연령이 높아질수록 신장기능이 저하되어 이...


In [38]:
import datetime
from PIL import ImageFont, ImageDraw, Image
import cv2
from ultralytics import YOLO


CONFIDENCE_THRESHOLD = 0.8
GREEN = (0, 255, 0)
WHITE = (255, 255, 255)
RED = (255, 0, 0)

classcode = ['K-001562', 'K-001901', 'K-002052', 'K-004378', 'K-005391', 'K-006192', 'K-006563', 'K-006697', 'K-007024', 'K-007348', 'K-012769', 'K-013398', 'K-013900', 'K-015710', 'K-016206', 'K-016548', 'K-016688', 'K-021771', 'K-023933', 'K-024029', 'K-024127', 'K-025629', 'K-027777', 'K-028408', 'K-031490', 'K-038104', 'K-038722', 'K-040265', 'K-041149', 'K-044830', 'K-044834']
classname_en = ['Minocin Cap. 50mg', 'Buspar Tab. 10mg Boryung', 'Dinacs Tab.', 'Tylenol Tab. 500mg', 'Proscar Tab. 5mg', 'Beecom-CF Tab.', 'Joins Tab.', 'Ssiznal Tab.', 'Festal Plus Tab.', 'Riboteine Tab.', 'Dr. Bearse Tab.', 'Isotinone Soft Cap.', 'Ebixa Tab.', 'Entelon Tab. 150mg', 'Avodart Soft Cap. 0.5mg', 'Gabatopa Tab. 100mg', 'Omacor Soft Cap.', 'Rabiet Tab.20mg', 'Allerdin Tab.', 'Neotigason Cap. 10mg', 'Cacepin Tab 25mg', 'Zeromine Soft Cap.', 'Kanarb Tab. 60mg', 'Olzapine Tab. 2.5mg', 'Aspid Tab.', 'Hanmi Tams Cap. 0.4mg', 'Entresto Film Coated Tab. 100mg', 'Agen-F Tab.', 'Ramnos Cap. 500mg', 'Haben Herb F Cap.', 'Elsteine Tab.']


model = YOLO('./weights/last.pt')

image = './DATASET/연령별 금기.png'
frame = cv2.imread(image)
detection = model(image)[0]

drug_list = []
boxes_dict = {}
ord_num = 1
for data in detection.boxes.data.tolist():  # data : [xmin, ymin, xmax, ymax, confidence_score, class_id]
    confidence = (data[4] * 100) / 2
    if confidence < CONFIDENCE_THRESHOLD:
        continue

    x1, y1, x2, y2 = int(data[0]), int(data[1]), int(data[2]), int(data[3])
    cls = int(data[5])

    drug_list.append(classcode[cls])
    boxes_dict[ord_num] = [cls, classcode[cls], (x1, y1), (x2, y2), confidence]
    ord_num += 1

    cv2.rectangle(frame, (x1, y1), (x2, y2), GREEN, 2)
    cv2.putText(frame, classname_en[cls]+' ', (x1, y1-5), cv2.FONT_ITALIC, 1, WHITE, 2)

cv2.imwrite("./temp/연령별금기.jpg", frame)

print(boxes_dict)
font = ImageFont.truetype('./font/NanumGothic.ttf', 18)
image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
image = Image.fromarray(image)
draw = ImageDraw.Draw(image)

for i in range(1, len(boxes_dict)+1) :
    drug_N = boxes_dict[i][1]
    point1 = boxes_dict[i][2]
    point2 = boxes_dict[i][3]

    print(point1)
    drug1 = df_img.loc[df_img['drug_N']==drug_N, 'drug_code'].values[0]   # K-xxx --> 648101510
    result = df_age_bann.loc[df_age_bann['제품코드']==drug1, ['성분명', '특정연령', '특정연령단위-2', '상세정보']]

    if len(result) >= 1 :
        element = result['성분명'].values[0]
        age = result['특정연령'].values[0]
        bound = result['특정연령단위-2'].values[0]
        alert_msg = result['상세정보'].values[0]

        alert_msg = alert_msg.replace(')', ')\n')

        draw.rectangle((point1, point2), outline=(255, 0, 0), width=3)
        draw.text((point1), element+'\n'+str(age)+bound+'\n'+alert_msg, font=font, fill=(255, 0, 0, 0))
    
    print(drug_N, len(result))


image.save('./temp/연령별금기.jpg')


image 1/1 c:\Users\tomozart21\VS_PROJECT\PILL_RnE\DATASET\ .png: 640x512 1 K-001562, 1 K-006697, 1 K-013398, 1 K-016206, 1 K-016688, 1 K-024029, 58.5ms
Speed: 1.0ms preprocess, 58.5ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 512)


{1: [11, 'K-013398', (618, 21), (861, 244), 48.22821319103241], 2: [14, 'K-016206', (536, 762), (832, 947), 47.93717563152313], 3: [16, 'K-016688', (157, 928), (307, 1230), 47.66554534435272], 4: [0, 'K-001562', (312, 465), (704, 678), 47.65027463436127], 5: [19, 'K-024029', (0, 60), (304, 347), 47.46555685997009], 6: [7, 'K-006697', (31, 465), (229, 685), 47.41392433643341]}
(618, 21)
K-013398 0
(536, 762)
K-016206 1
(157, 928)
K-016688 0
(312, 465)
K-001562 1
(0, 60)
K-024029 0
(31, 465)
K-006697 2


In [135]:
drug_code = df_img.loc[df_img['drug_N'].isin(drug_list), 'drug_code'].to_list()   # K-xxx --> 648101510

df_age_bann = df_age_bann.loc[df_age_bann['제품코드'].isin(drug_code), ['제품코드', '성분명', '특정연령', '상세정보']]

In [137]:
df_age_bann

,제품코드,성분명,특정연령,상세정보
858,644700590,minocycline hydrochloride,12,"(다만, 다른 약을 사용할 수 없거나 효과가 없는 경우에만 8세 이상 신중투여) 소..."
1604,650000780,dutasteride,18,안전성 및 유효성 미확립
2511,643601250,cetirizine dihydrochloride,6,안전성 및 유효성 미확립
3411,643601250,cetirizine dihydrochloride,6,안전성 및 유효성 미확립


In [11]:
import pandas as pd

df = pd.read_csv('./DRUG_INFO/효능군중복주의약물.csv')
df_effect = df[['약품코드', '효능군', '일반명', '품목명', '업체명']]

df_effect

,약품코드,효능군,일반명,품목명,업체명
0,652100060,당뇨병용제,acarbose,글루코바이정100밀리그람(아카보즈)_(0.1g/1정),(주)한독
1,652100050,당뇨병용제,acarbose,글루코바이정50밀리그람(아카보즈)_(50mg/1정),(주)한독
2,52401220,호흡기관용약,acebrophylline,아이프린캡슐(아세브로필린)_(0.1g/1캡슐),(주)인트로바이오파마
3,54801220,호흡기관용약,acebrophylline,아케로캡슐(아세브로필린)_(0.1g/1캡슐),엔비케이제약(주)
4,73000090,호흡기관용약,acebrophylline,아로필립캡슐100mg(아세브로필린)_(0.1g/1캡슐),건일바이오팜주식회사
...,...,...,...,...,...
15324,644806640,호흡기관용약,pseudoephedrine,노즈코프연질캡슐_(1캡슐),태극제약(주)
15325,658108450,호흡기관용약,chlorpheniramine,지민에스정_(1정),(유)한풍제약
15326,674402520,호흡기관용약,Pseudoephedrine,큐어코플러스연질캡슐_(1캡슐),아이큐어(주)
15327,674402520,해열진통소염제,acetaminophen,큐어코플러스연질캡슐_(1캡슐),아이큐어(주)


In [7]:
df.columns

Index(['효능군', '그룹구분', '일반명코드', '일반명', '효능군중복점검코드', '약품코드', '품목명', '업체명', '급여',
       '공고일자', '공고번호'],
      dtype='object')

In [8]:
df.head(2)

,효능군,그룹구분,일반명코드,일반명,효능군중복점검코드,약품코드,품목명,업체명,급여,공고일자,공고번호
0,당뇨병용제,Group 3,100601ATB,acarbose,396C0001,652100060,글루코바이정100밀리그람(아카보즈)_(0.1g/1정),(주)한독,급여,2020-09-24,20200423
1,당뇨병용제,Group 3,100602ATB,acarbose,396C0001,652100050,글루코바이정50밀리그람(아카보즈)_(50mg/1정),(주)한독,급여,2020-09-24,20200423
